In [ ]:
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sec_certs.dataset.fips import FIPSDataset
import pandas as pd
import seaborn as sns
import numpy as np
from pysankey import sankey

%matplotlib inline

matplotlib.use("pgf")
sns.set_theme(style='white')
plt.rcParams["axes.linewidth"] = 0.5
plt.rcParams["legend.fontsize"] = 6.5
plt.rcParams["xtick.labelsize"] = 8
plt.rcParams["ytick.labelsize"] = 8
plt.rcParams["ytick.left"] = True
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['ytick.major.pad'] = 0
plt.rcParams["xtick.bottom"] = True
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['xtick.major.pad'] = 0
plt.rcParams["pgf.texsystem"] = "pdflatex"
plt.rcParams["font.family"] = "serif"
plt.rcParams["text.usetex"] = True
plt.rcParams["pgf.rcfonts"] = False
plt.rcParams["axes.titlesize"] = 8
plt.rcParams["legend.handletextpad"] = 0.3
plt.rcParams['lines.markersize'] = 4
plt.rcParams['savefig.pad_inches'] = 0.01
sns.set_palette("deep")

#plt.style.use("seaborn-whitegrid")
#sns.set_palette("deep")
#sns.set_context("notebook")  # Set to "paper" for use in paper :)

#plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
dset = FIPSDataset.from_web_latest()

In [ ]:
df = dset.to_pandas()

In [ ]:
df.head()